<a href="https://colab.research.google.com/github/yilinearity/Kinetic_Fit_Simulation/blob/main/BiMol_Rev_FitABC_v1_3_GColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fitting Kinetic Data to a + b = c (Reversible)**

```
d[a]/dt = -k1 * [a] * [b] + k2 * [c]
d[b]/dt = -k1 * [a] * [b] + k2 * [c]
d[c]/dt = +k1 * [a] * [b] - k2 * [c]
rate constant = k1 and k2 (forward and reverse)
```

The input data should be in a csv format, with four columns **Time, [A], [B], and [C]**. The first row is the heading row.

## **To Use**
The script has two cells; <font color='blue'>click on the **run** icon (on the left side of the code) to run each cell.</font>

In the first cell, the data csv file will be first imported (uploaded), and a slider will show up to select a good initial value of ***k_initial***. Note that this ***k_initial*** is the exponent of ***k*** (i.e. *k* = 10^(***k_initial***)). The current ranges are set as (-10.0, 5.0) and (-20.0, 5.0); change them to suit your need. A good initial value for ***k*** will ensure the successful fit of the data set.

The value of ***k_initial*** is then used in the second cell to complete the fitting routine, which made use of `odeint` from `scipy.integrate`. Finally, the results are saved and exported as a csv (in Google Drive); the rate constant ***k*** will be displayed on the screen. Modify the path of the Google Drive folder if needed. (Be patient, the saving takes time on Google Drive.)

<br>

---
MIT License Copyright (c) 2023 Yi-Lin Wu (yilinearity@gmail.com) <br>
Full text of the MIT License can be found at: https://opensource.org/licenses/MIT

In [ ]:
import os
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from ipywidgets import interactive

# Define the system of ODEs
def system(y, t, k1, k2):
    a, b, c = y
    dadt = -k1 * a * b + k2 * c
    dbdt = -k1 * a * b + k2 * c
    dcdt = k1 * a * b - k2 * c
    return [dadt, dbdt, dcdt]

# Function to upload and process the CSV file
def process_uploaded_file(uploaded_file):
    data = pd.read_csv(uploaded_file)
    t = data.iloc[:, 0]
    a = data.iloc[:, 1]
    b = data.iloc[:, 2]
    c = data.iloc[:, 3]
    return t, a, b, c

# Initialize the k variables as global variables
k1 = 10**(-3)
k2 = 10**(-5)

def plot(k1_initial=-3, k2_initial=-5):
    global k1, k2  # Declare k1 and k2 as global variables
    k1 = 10**k1_initial  # Update the global k1 value
    k2 = 10**k2_initial  # Update the global k2 value

    # Check if the data is available
    if 't' not in globals() or 'a' not in globals() or 'b' not in globals() or 'c' not in globals():
        print("Please upload a CSV file first.")
        return

    # Solve the system of ODEs with the global k values
    sol = odeint(system, [a[0], b[0], c[0]], t, args=(k1, k2))
    a_fit = sol[:, 0]
    b_fit = sol[:, 1]
    c_fit = sol[:, 2]

    # Plot the results
    plt.plot(t, a, 'o', label='a (data)')
    plt.plot(t, b, 'o', label='b (data)')
    plt.plot(t, c, 'o', label='c (data)')
    plt.plot(t, a_fit, label='a (fit)')
    plt.plot(t, b_fit, label='b (fit)')
    plt.plot(t, c_fit, label='c (fit)')
    plt.legend()
    plt.show()

    return k1_initial, k2_initial

# Use Colab's file upload feature to upload the CSV file
from google.colab import files
uploaded = files.upload()

# Process the uploaded file
for filename in uploaded.keys():
    t, a, b, c = process_uploaded_file(filename)

# Create the interactive plot widget
w = interactive(plot, k1_initial=(-10.0, 5.0), k2_initial=(-20.0, 5.0))
w


In [ ]:
from google.colab import drive

# Confirm the result from the 1st section is passed down here
k1_exp, k2_exp = w.kwargs['k1_initial'], w.kwargs['k2_initial']
print(f"k1_exp = {k1_exp}, k2_exp = {k2_exp}")

print(f"The initial value of k1 is: {10**k1_exp:.3f}")
print(f"The initial value of k2 is: {10**k2_exp:.6f}")

# Define the function to fit k's
def fit_k(x, k1, k2):
    sol, info = odeint(system, [a[0], b[0], c[0]], t, args=(k1,k2), full_output=True)
    return (sol[:, 0] - a)**2 + (sol[:, 1] - b)**2 + (sol[:, 2] - c)**2

# Find the optimal value of k using the selected initial value, the method can be lm, trf, or dogbox
k1_initial = 10**k1_exp
k2_initial = 10**k2_exp
popt, pcov = curve_fit(fit_k, xdata=None, ydata=np.zeros_like(t), p0=[k1_initial,k2_initial], method='lm')
k1, k2 = popt
perr = np.sqrt(np.diag(pcov))

# Solve the system of ODEs with the optimal value of k's
sol = odeint(system, [a[0], b[0], c[0]], t, args=(k1, k2))
a_fit = sol[:, 0]
b_fit = sol[:, 1]
c_fit = sol[:, 2]

# Plot the results
plt.plot(t, a, 'o', label='a (data)')
plt.plot(t, b, 'o', label='b (data)')
plt.plot(t, c, 'o', label='c (data)')
plt.plot(t, a_fit, label='a (fit)')
plt.plot(t, b_fit, label='b (fit)')
plt.plot(t, c_fit, label='c (fit)')
plt.legend()
plt.show()

# Display the results
print(f"k1 = {k1:.4e} ± {perr[0]:.4e}")
print(f"k2 = {k2:.4e} ± {perr[1]:.4e}")

output = pd.DataFrame({'t': t, 'a': a, 'b': b, 'c': c, 'a_fit': a_fit, 'b_fit': b_fit, 'c_fit': c_fit})
print(output)

# Save the DataFrame to a CSV file in Google Drive
from google.colab import drive
drive.mount('/content/drive')
output.to_csv('/content/drive/My Drive/results/BiMol_Rev_Fit_ABC.csv', sep=",", index=False)